# Setup

In [20]:
# import python packages
import kfp
import kfp.dsl as dsl
from kfp import compiler
from kfp import components

In [21]:
# setup the values
DOCKER_IMAGE = "tensorflow/tensorflow:2.0.0b0-py3"
EXPERIMENT_NAME = "test kubeflow"
PIPELINE_FILE_NAME = "test_pipeline.yaml" 

# Create Pipeline

パイプラインを構築します。<br>
今回は単純な加算処理を行うパイプラインとなります。

[こちら](https://github.com/kubeflow/examples/tree/master/pipelines/simple-notebook-pipeline)のkubeflowのサンプルコードを参考にしています。

In [22]:
@dsl.python_component(
    name='add_op',
    description='adds two numbers',
    base_image=DOCKER_IMAGE 
)
def add(a: float, b: float) -> float:
    '''Calculates sum of two arguments'''
    print(a, '+', b, '=', a + b)
    return a + b

add_op = components.func_to_container_op(
    add,
    base_image=DOCKER_IMAGE, 
)

/var/folders/w2/ync7d1s96d3gsjx8f_17f3f9hbtnhx/T/ipykernel_3784/3218079999.py:1: DeprecationWarning: Call to deprecated function (or staticmethod) python_component. (This decorator does not seem to be used, so we deprecate it. If you need this decorator, please create an issue at https://github.com/kubeflow/pipelines/issues) -- Deprecated since version 0.2.6.
  @dsl.python_component(


In [23]:

@dsl.pipeline(
   name='Calculation pipeline',
   description='A toy pipeline that performs arithmetic calculations.'
)
def calc_pipeline(
   a: float =0,
   b: float =7
):
    #Passing pipeline parameter and a constant value as operation arguments
    add_task = add_op(a, 4) #Returns a dsl.ContainerOp class instance. 
    
    #You can create explicit dependency between the tasks using xyz_task.after(abc_task)
    add_2_task = add_op(a, b)
    
    add_3_task = add_op(add_task.output, add_2_task.output)

# Compile Pipeline

構築したパイプラインをyamlにコンパイル（書き込み）します。

In [25]:
kfp.compiler.Compiler().compile(
    pipeline_func=calc_pipeline,
    package_path=PIPELINE_FILE_NAME
)